In [ ]:
# TODOS
# Add Rician objective function for LSQ fitting.
# rewrite for experiment

"""
(c) Stefano B. Blumberg and Paddy J. Slator, do not redistribute or modify

Code to replicate the ADC experiment (alongside matlab code - maybe translate to python?) <Add paper link>

Overview for cells:
    - Choose data size splits 2
    - Generate data examples 3-A/B/C
    - Data format for JOFSTO 4
    - Option to pass data directly, or save to disk and load 5-A/B
    - JOFSTO hyperparameters 6,7,8
"""

In [ ]:
########## (1)
# Import modules, see requirements.txt for jofsto requirements, set global seed

import numpy as np
import os
import sys
sys.path.append(os.path.abspath('..'))
import os, yaml 
from jofsto_code.jofsto_main import return_argparser, run

import matplotlib.pyplot as plt

np.random.seed(0)  # Random seed for entire script

In [ ]:
#Directories and filenames to save data (Replace with location of JOFSTO code - possible to get this automatically?)
basedir = '/Users/paddyslator/python/ED_MRI/'

In [ ]:
########## (2)
# Data split sizes

n_train = 100000  # No. training voxels, reduce for faster training speed
n_val = n_train // 10  # No. validations set voxels
n_test = n_train // 10  # No. test set voxels

n_samples = n_train + n_val + n_test #total number of samples to simulate

#choose the size of the super-design
C_bar = 192


In [ ]:
#define some models and generate the data

model_name = 't1inv'

if model_name == 'adc':
    #model equation for simulation
    def model(D,bvals):
        signals = np.exp(-bvals*D)
        return signals
    
    #min/max parameter values
    minD = 0.1
    maxD = 3
    
    #simulate parameter values
    parameters = np.random.uniform(low=minD,high=maxD,size=(n_samples,1))

    #Generate data using the model
    
    #make the super design
    maxb = 5
    minb = 0
    acq_params_super = np.linspace(minb,maxb,C_bar)
    
    #generate the data    
    raw_signals = np.zeros((n_samples,C_bar),dtype = np.float32)
    for i in range(0,n_samples):
        raw_signals[i,:] = model(parameters[i],acq_params_super)
    
    
elif model_name == 't1inv':
    def model(T1,ti,tr):
        signals = abs(1 - (2*np.exp(-ti/T1)) + np.exp(-tr/T1))
        return signals
    
    #min/max parameter values
    minT1 = 0.1
    maxT1 = 7
    #simulate parameter values
    parameters = np.random.uniform(low=minT1,high=maxT1,size=(n_samples,1))
    
    #generate data using an T1 inversion recovery model 
    
    #make the super design
    tr = 7 #repetition time
    maxti = tr
    minti = 0.1
    acq_params_super = np.linspace(minti,maxti,C_bar)
    
    #generate the data
    raw_signals = np.zeros((n_samples,C_bar),dtype = np.float32)
    for i in range(0,n_samples):
        raw_signals[i,:] = model(parameters[i],acq_params_super,tr)

    




In [ ]:
########## (3-A)
# Create dummy, randomly generated (positive) data

# C_bar = 220
# M = 12  # Number of input measurements \bar{C}, Target regressors
# rand = np.random.lognormal  # Random genenerates positive
# train_inp, train_tar = rand(size=(n_train, C_bar)), rand(size=(n_train, M))
# val_inp, val_tar = rand(size=(n_val, C_bar)), rand(size=(n_val, M))
# test_inp, test_tar = rand(size=(n_test, C_bar)), rand(size=(n_test, M))


# #########






    
    
    






In [ ]:
#add noise to the data
def add_noise(data, scale=0.05):
    data_real = data + np.random.normal(scale=scale, size=np.shape(data))
    data_imag = np.random.normal(scale=scale, size=np.shape(data))
    data_noisy = np.sqrt(data_real**2 + data_imag**2)

    return data_noisy


SNR = 20
signals = add_noise(raw_signals,1/SNR)




In [ ]:
########## (4)
# Load data into JOFSTO format

# Data in JOFSTO format, \bar{C} measurements, M target regresors
data = dict(
    train=signals[0:n_train,:],  # Shape n_train x \bar{C}
    train_tar=parameters[0:n_train,:],  # Shape n_train x M
    val=signals[n_train:(n_train + n_val),:],  # Shape n_val x \bar{C}
    val_tar=parameters[n_train:(n_train + n_val),:],  # Shape n_val x M
    test=signals[(n_train + n_val):(n_train + n_val + n_test),:],  # Shape n_test x \bar{C}
    test_tar=parameters[(n_train + n_val):(n_train + n_val + n_test),:],  # Shape n_test x M
)

#with open(os.path.dirname(__file__) + "/base.yaml", "r") as f:
#with open("/home/blumberg/Bureau/z_Automated_Measurement/Code/base.yaml", "r") as f:
with open(os.path.join(basedir, "base.yaml"), "r") as f:
    jofsto_args =  yaml.safe_load(f)

In [ ]:
########## (5-A)
# Option to save data to disk, and JOFSTO load

# Check whether a path for this model exists or not - if not create it
data_dir = os.path.join(basedir,'output',model_name + '_simulations')
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    

proj_params =  '_simulations_' + 'n_train_' + str(n_train) + '_SNR_' + str(SNR)
    
data_fil = os.path.join(data_dir, model_name + proj_params +  '.npy')
#data_fil = "/home/blumberg/Bureau/z_Automated_Measurement/Output/paddy/adc_simulations.npy"
#data_fil = "/Users/paddyslator/python/ED_MRI/adc_simulations.npy"  # Add path to save file
np.save(data_fil, data)
print("Saving data as", data_fil)
pass_data = None

jofsto_args["--data_fil"] = data_fil


########## (5-B)
# Option to pass data to JOFSTO directly

pass_data = data

In [ ]:
########## (6)
# Simplest version of JOFSTO, modifying the most important hyperparameters


# Decreasing feature subsets sizes for JOFSTO to consider
jofsto_args["jofsto_train_eval"]["C_i_values"] = [C_bar, C_bar // 2, C_bar // 4, C_bar // 8, C_bar // 16]

# Feature subset sizess for JOFSTO evaluated on test data
jofsto_args['jofsto_train_eval']["C_i_eval"] = [C_bar // 2, C_bar // 4, C_bar // 8, C_bar // 16]

# Scoring net C_bar -> num_units_score[0] -> num_units_score[1] ... -> C_bar units
jofsto_args['network']["num_units_score"] = [1000, 1000]

# Task net C_bar -> num_units_task[0] -> num_units_task[1] ... -> M units
jofsto_args['network']["num_units_task"] = [1000, 1000]

jofsto_args['output']["out_base"] =  data_dir  #"/Users/paddyslator/python/ED_MRI/test1" #"/home/blumberg/Bureau/z_Automated_Measurement/Output/paddy"
jofsto_args['output']["proj_name"] = model_name + proj_params
jofsto_args['output']["run_name"] = "test"

jofsto_args['other_options']['save_output'] = True

#jofsto_args["total_epochs"] = 1000

JOFSTO_output_small = run(args=jofsto_args, pass_data=pass_data)


In [ ]:
#load the FULL JOFSTO output
JOFSTO_output = np.load(os.path.join(basedir,jofsto_args['output']["out_base"],jofsto_args['output']["proj_name"],"results", jofsto_args['output']["run_name"] + "_all.npy"), allow_pickle=True).item()




In [ ]:
#extract some useful parameters fom the jofsto output
#final subset index
C_last = JOFSTO_output['args']['jofsto_train_eval']['C_i_eval'][-1]
#chosen acquisition parameters
acq_params_jofsto = acq_params_super[JOFSTO_output[C_last]['measurements']]

In [ ]:
from scipy.optimize import minimize

#define CRLB functions 

if model_name == 'adc':
    def f_crlb(b,params,sigma):
        #params[0] is S0
        #params[1] is ADC 
        #params = np.zeros(2)
        #params[0] = 1
        #params[1] = 1
        #sigma = 0.05  
        
        #need 2 b-values - so assume there is always a b=0 (CRLB with 2 b-values always chooses a b=0 anyway)
        b=np.insert(b,0,0)
                        
        dy = np.zeros((len(b),2))
        dy[:,0] = np.exp(-b * params[1])
        dy[:,1] = -b*params[0]*np.exp(-b*params[1])
       
        fisher = (np.matmul(dy.T,dy))/sigma**2

        invfisher = np.linalg.inv(fisher)
        #second diagonal element is the lower bound on the variance of the ADC
        f = invfisher[1,1]

        return f
elif model_name == 't1inv':
    def f_crlb(ti,params,tr,sigma):
        #params[0] is S0
        #params[1] is T1  
        #convert to R1
        params[1] = 1/params[1]
        #tr = 7
        #sigma = 1 

        dy = np.zeros((len(ti),2))
        dy[:,0] = (1 - 2*np.exp(-ti * params[1]) + np.exp(-tr*params[1])) 
        dy[:,1] = params[0]*(2*ti*np.exp(-ti*params[1]) -tr*np.exp(-tr*params[1]))
        
        fisher = (np.matmul(dy.T,dy))/sigma**2

        invfisher = np.linalg.inv(fisher)
        #second diagonal element is the lower bound on the variance of R1
        f = invfisher[1,1]

        return f


#calculate CRLB optimal acquisition parameter (e.g. b-value, TI) for a range of model parameters (e.g. ADC, T1)

#match number of model parameters in the range to the number of measurements in the final JOFSTO output


#Calculate the range of parameters
if model_name == 'adc': 
    params = np.linspace(0,maxD,C_last)[1:] #one less parameter for ADC as CRLB assumes a b=0
elif model_name == 't1inv':
    params = np.linspace(0,maxT1,C_last+1)[1:] 
    
#initialise array of CRLB-optimsed acquisition parameters (e.g. b-values, TI)
acq_params_crlb = np.zeros(C_last)

#these don't affect the optimisation so can be fixed
S0 = 1
sigma = 1/SNR

for i in range(0,len(params)):
    if model_name == 'adc':
        fixed_args = (np.array((S0,params[i])),sigma)
        bnds=((minb,maxb),)
        init = 1/params[i]
#        init = np.array((1/params[i],2/params[i]))
    elif model_name == 't1inv':
        fixed_args = (np.array((S0,params[i])), tr, sigma)
        bnds=((minti,maxti),)
        init = params[i]
        
    acq_params_crlb[i] = minimize(f_crlb, init, args=fixed_args, method='Nelder-Mead',bounds=bnds).x

        

In [ ]:
# #calculate robust CRLB by summing over many possible D/T1's
# #as in McHugh et al. MRM 2018 doi: 10.1002/mrm.27551

# #take the first 100 parameters (as in McHugh)
# parameters_robust = parameters[0:100]

# def f_crlb_robust(b,S0,parameters_robust,sigma):
#     for param in parameters_robust:
#         f_summand = f_crlb(b,np.array((S0,param),dtype=object),sigma)
        
#     return np.sum(np.log(f_summand))


# if model_name == 'adc':
#     fixed_args = (S0,parameters_robust,sigma)
#     bnds=((minb,maxb),)
#     init = 1/params[i]
#     init = np.array((1/params[i],2*params[i],3*params[i]))
# elif model_name == 't1inv':
#     fixed_args = (S0,parameters_robust, tr, sigma)
#     bnds=((minti,maxti),)
#     init = params[i]
        

# acq_params_CRLB_robust = minimize(f_crlb_robust, np.linspace(minb,maxb,C_last), args=fixed_args, method='Nelder-Mead',bounds=bnds).x







In [ ]:
#plot the JOFSTO and CRLB acquisition parameters (e.g. b-values, TI,...)
#all super-design acquisition parameters
paramtest = params[5]

if model_name == 'adc':
    #all super-design b-values
    plt.plot(acq_params_super,model(paramtest,acq_params_super),'k.',markersize=2)
    #JOFSTO chosen b-values
    C_last = JOFSTO_output['args']['jofsto_train_eval']['C_i_eval'][-1]
    plt.plot(acq_params_jofsto, model(paramtest, acq_params_jofsto),'bs',fillstyle='none',markeredgewidth=2)
    #CRLB chosen b-values
    plt.plot(acq_params_crlb,model(paramtest,acq_params_crlb),'rs',fillstyle='none')
    plt.legend(['super-design','JOFSTO','CRLB'],fontsize=14)
    plt.xlabel('b-value ($\mu$m$^2$ s$^{-1}$)',fontsize=14)
elif model_name == 't1inv':
    #all super-design b-values
    plt.plot(acq_params_super,model(paramtest,acq_params_super,tr),'k.',markersize=2)
    #JOFSTO chosen b-values
    C_last = JOFSTO_output['args']['jofsto_train_eval']['C_i_eval'][-1]
    plt.plot(acq_params_jofsto, model(paramtest,acq_params_jofsto,tr),'bs',fillstyle='none',markeredgewidth=2)
    #CRLB chosen b-values
    plt.plot(acq_params_crlb,model(paramtest,acq_params_crlb,tr),'r.',markeredgewidth=3)
    plt.legend(['Super design','JOFSTO','CRLB'],fontsize=14)
    plt.xlabel('Inversion time (s)',fontsize=14)


plt.ylabel('Signal',fontsize=14)
    
#define location to save figures
fig_dir = os.path.join(basedir,'figures')
# Check whether a path for this model exists or not - if not create it
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)
    
#base filename for saving figures    
fig_basename = model_name + '_simulations_n_train_' + str(n_train) + '_' + str(SNR)

plt.savefig(os.path.join(fig_dir,fig_basename + '_acq_params.png'),dpi=300)
plt.savefig(os.path.join(fig_dir,fig_basename + '_acq_params.eps'),dpi=300)
plt.savefig(os.path.join(fig_dir,fig_basename + '_acq_params.pdf'),dpi=300)


print('TO DO: move this to the plotting function and plot for all SNR. In text: "we examine the chosen acquisition schemes for the simple models"')



In [ ]:
#helper functions for model fitting 

def rician_log_likelihood(signals,synth_signals,sigma):
    sumsqsc = (signals**2 + synth_signals**2)/(2 * sigma**2)
#    print("sumsqsc: " + str(sumsqsc))
    scp = synth_signals * signals / sigma**2  
#    print("scp: " + str(scp))   
#    lb0 = np.log(np.i0(scp))
    lb0 = log_i0(scp)
#    print("lb0: " + str(lb0))   
    log_likelihoods = -2*np.log(sigma) - sumsqsc + np.log(synth_signals) + lb0
#    print("log_likelihoods: " + str(log_likelihoods))   


    return np.sum(log_likelihoods)

if model_name == 'adc':
    def rician_objective_function(D,bvals,signals,sigma):
        return -rician_log_likelihood(model(D,bvals),signals,sigma)

    def gaussian_objective_function(D,bvals,signals):
        return np.mean((signals - model(D,bvals))**2)
elif model_name == 't1inv':
    def rician_objective_function(T1,ti,tr,signals,sigma):
        return -rician_log_likelihood(model(T1,ti,tr),signals,sigma)

    def gaussian_objective_function(T1,ti,tr,signals):
        return np.mean((signals - model(T1,ti,tr))**2)



def log_i0(x):
    exact = x < 700
    approx = x >= 700
   
    lb0 = np.zeros(np.shape(x))
    lb0[exact] = np.log(np.i0(x[exact]))
    #This is a more standard approximation.  For large x, I_0(x) -> exp(x)/sqrt(2 pi x).
    lb0[approx] = x[approx] - np.log(2*np.pi*x[approx])/2
    
    return lb0
    


In [ ]:
from scipy.optimize import minimize

#fit the model on the super-design acquisition, JOFSTO acquisition, CRLB acquisition

#choose the noise level
sigma_test = 1/SNR #set to the level of the training data simulations
#sigma_test = 0.2 #set to something else


#for the super design use the test data signals - but with different noise 
raw_signals_super = signals[(n_train + n_val):(n_train + n_val + n_test),:]
signals_super = add_noise(raw_signals_super,sigma_test)

#for jofsto use the reconstructed data
signals_jofsto = signals[(n_train + n_val):(n_train + n_val + n_test),JOFSTO_output[C_last]['measurements']] 


#for CRLB simulate data for the CRLB acquisition - as don't have test data at these b-values
signals_crlb = np.zeros((n_test,C_last))
#use the ground truth parameters from the test dataset
gt_parameters = data["test_tar"][:,0]
for i in range(0,n_test):
    if model_name == 'adc':
        signals_crlb[i,:] = add_noise(model(gt_parameters[i],acq_params_crlb),scale=sigma_test)
    if model_name == 't1inv':
        signals_crlb[i,:] = add_noise(model(gt_parameters[i],acq_params_crlb,tr),scale=sigma_test)

    

#choose starting parameter value for the fit - this should work fine for both models
paramstart = 2

fitted_parameters_crlb = np.zeros(n_test)
fitted_parameters_super = np.zeros(n_test)
fitted_parameters_jofsto = np.zeros(n_test)


#fit the models to the data
#Note that we pass the ground truth sigma to the fitting methods - but JOFSTO_NN never sees the ground truth sigma
for i in range(0,n_test):
    if model_name == 'adc':
        fitted_parameters_crlb[i] = minimize(rician_objective_function, paramstart, args=(acq_params_crlb,signals_crlb[i,:],sigma_test),method='Nelder-Mead').x
        fitted_parameters_super[i] = minimize(rician_objective_function, paramstart, args=(acq_params_super,signals_super[i,:],sigma_test),method='Nelder-Mead').x
        fitted_parameters_jofsto[i] = minimize(rician_objective_function, paramstart, args=(acq_params_jofsto,signals_jofsto[i,:],sigma_test),method='Nelder-Mead').x
    if model_name == 't1inv':
        fitted_parameters_crlb[i] = minimize(rician_objective_function, paramstart, args=(acq_params_crlb,tr,signals_crlb[i,:],sigma_test),method='Nelder-Mead').x
        fitted_parameters_super[i] = minimize(rician_objective_function, paramstart, args=(acq_params_super,tr,signals_super[i,:],sigma_test),method='Nelder-Mead').x
        fitted_parameters_jofsto[i] = minimize(rician_objective_function, paramstart, args=(acq_params_jofsto,tr,signals_jofsto[i,:],sigma_test),method='Nelder-Mead').x
        
  

    
#extract the JOFSTO nn fit
fitted_parameters_jofsto_nn = JOFSTO_output[jofsto_args["jofsto_train_eval"]["C_i_values"][-1]]["test_output"][:,0]





In [ ]:
#save the ground truth parameters, signals, and fitted parameters for plotting later (ground truth is saved as "data" above)

#crlb test signals
#np.save(os.path.splitext(data_fil)[0] + '_signals_crlb.npy', signals_crlb) 
np.save(os.path.join(data_dir, model_name + proj_params, 'signals_crlb.npy'), signals_crlb)

#super design test signals - these are already saved as "data" dictionary above but also save here for neatness
#np.save(os.path.splitext(data_fil)[0] + '_signals_super.npy', signals_super) 
np.save(os.path.join(data_dir, model_name + proj_params, 'signals_super.npy'), signals_super)

#jofsto recon test signals - also saved in the jofsto output but also save separately for neatness
#np.save(os.path.splitext(data_fil)[0] + '_signals_jofsto.npy', signals_jofsto) 
np.save(os.path.join(data_dir, model_name + proj_params, 'signals_jofsto.npy'), signals_jofsto)


#save the lsq fits
#np.save(os.path.splitext(data_fil)[0] + '_fit_crlb.npy', fitted_parameters_crlb) 
np.save(os.path.join(data_dir, model_name + proj_params, 'fit_crlb.npy'), fitted_parameters_crlb)

#np.save(os.path.splitext(data_fil)[0] + '_fit_super.npy', fitted_parameters_super) 
np.save(os.path.join(data_dir, model_name + proj_params, 'fit_super.npy'), fitted_parameters_super)

#np.save(os.path.splitext(data_fil)[0] + '_fit_jofsto_lsq.npy', fitted_parameters_jofsto) 
np.save(os.path.join(data_dir, model_name + proj_params, 'fit_jofsto_lsq.npy'), fitted_parameters_jofsto)


#save the jofsto nn fits
#np.save(os.path.splitext(data_fil)[0] + '_fit_jofsto_nn.npy', fitted_parameters_jofsto_nn) 
np.save(os.path.join(data_dir, model_name + proj_params, 'fit_jofsto_nn.npy'), fitted_parameters_jofsto_nn)


#save the test dataset ground truth parameters
#np.save(os.path.splitext(data_fil)[0] + '_parameters_gt.npy', gt_parameters) 
np.save(os.path.join(data_dir, model_name + proj_params, 'parameters_gt.npy'), gt_parameters)


    
    
    
#super design full dataset signals - these are already saved as "data" dictionary above but also save here for neatness
#np.save(os.path.splitext(data_fil)[0] + '_signals_super_full.npy', signals) 
np.save(os.path.join(data_dir, model_name + proj_params, 'signals_super_full.npy'), signals)


#save the full dataset ground truth parameters 
#np.save(os.path.splitext(data_fil)[0] + '_parameters_gt_full.npy', parameters) 
np.save(os.path.join(data_dir, model_name + proj_params, 'parameters_gt_full.npy'), parameters)

#save the super design acquisition parameters
#np.save(os.path.splitext(data_fil)[0] + '_acq_params_super.npy', acq_params_super)
np.save(os.path.join(data_dir, model_name + proj_params, 'acq_params_super.npy'), acq_params_super)

#save the CRLB acquisition parameters
#np.save(os.path.splitext(data_fil)[0] + '_acq_params_super.npy', acq_params_super)
np.save(os.path.join(data_dir, model_name + proj_params, 'acq_params_crlb.npy'), acq_params_crlb)

#save the JOFSTO acquisition parameters
np.save(os.path.join(data_dir, model_name + proj_params, 'acq_params_jofsto.npy'), acq_params_jofsto)



    

In [ ]:
########## (7)
# Modify more JOFSTO hyperparameters, less important, may change results


In [ ]:

########## (8)
# Deep learning training hyperparameters for inner loop